In [6]:
import psycopg2

conn = psycopg2.connect(database = "postgres",
                        user = "postgres",
                        port = 5433)
cur = conn.cursor()

In [16]:
cur.execute("SELECT * FROM department")
print(cur.fetchall())
conn.commit()

[(1, 'Business'), (2, 'Analytics')]


In [62]:
# create table
cur = conn.cursor()
# cur.execute(
# '''
# CREATE TABLE EXAMPLE1 (
#                         NAME VARCHAR(100) PRIMARY KEY,
#                         DEPARTMENT VARCHAR(100),
#                         SALARY INT
#                       );
# ''')

cur.execute("""
CREATE TABLE EXAMPLE2 (DATE DATE,
                       REVENUE INT);
""")
conn.commit()
cur.close()

In [63]:
# add data

# query = """
# INSERT INTO example1 (NAME, DEPARTMENT, SALARY)
# VALUES
#     ('John', 'Sales', 5000),
#     ('Alice', 'Sales', 5500),
#     ('Bob', 'Sales', 4800),
#     ('Mary', 'Marketing', 6000),
#     ('Tom', 'Marketing', 6200),
#     ('Emily', 'Marketing', 5800);
# """

# cur = conn.cursor()
# cur.execute(query)
# conn.commit()
# cur.close()

query = """
INSERT INTO example2 (date, revenue)
VALUES
    ('2021-01-01', 1000),
    ('2021-01-02', 1500),
    ('2021-01-03', 1200),
    ('2021-01-04', 2000);
"""

cur = conn.cursor()
cur.execute(query)
conn.commit()
cur.close()

In [23]:
# drop table
cur = conn.cursor()
cur.execute('DROP TABLE EXAMPLE1')
conn.commit()
cur.close()

In [61]:
query = """
SELECT 
    NAME,
    DEPARTMENT,
    salary,
    row_number() over (partition by DEPARTMENT) as rank
from 
    EXAMPLE1
"""

cur = conn.cursor()
cur.execute(query)
cur.fetchall()

[('Mary', 'Marketing', 6000, 1),
 ('Tom', 'Marketing', 6200, 2),
 ('Emily', 'Marketing', 5800, 3),
 ('John', 'Sales', 5000, 1),
 ('Alice', 'Sales', 5500, 2),
 ('Bob', 'Sales', 4800, 3)]

In [72]:
query = """
SELECT 
    DATE,
    REVENUE,
    REVENUE - lag(revenue) over () as revenue_change 
FROM 
    EXAMPLE2
"""

cur = conn.cursor()
cur.execute(query)
cur.fetchall()

[(datetime.date(2021, 1, 1), 1000, None),
 (datetime.date(2021, 1, 2), 1500, 500),
 (datetime.date(2021, 1, 3), 1200, -300),
 (datetime.date(2021, 1, 4), 2000, 800)]

In [75]:
conn.commit()
cur.close()

In [76]:
query = """
SELECT 
    DATE,
    REVENUE,
    AVG(revenue) over (rows between 1 preceding and 1 following)
FROM 
    EXAMPLE2
"""

cur = conn.cursor()
cur.execute(query)
cur.fetchall()

[(datetime.date(2021, 1, 1), 1000, Decimal('1250.0000000000000000')),
 (datetime.date(2021, 1, 2), 1500, Decimal('1233.3333333333333333')),
 (datetime.date(2021, 1, 3), 1200, Decimal('1566.6666666666666667')),
 (datetime.date(2021, 1, 4), 2000, Decimal('1600.0000000000000000'))]

In [80]:
## Text Searches

In [79]:
query_create_table = """
-- Create the sample_text_data table
CREATE TABLE sample_text_data (
    id SERIAL PRIMARY KEY,
    text_content TEXT
);
"""

query_add_data = """
-- Insert sample text data
INSERT INTO sample_text_data (text_content) VALUES
('This is a sample text containing important information.'),
('The quick brown fox jumps over the lazy dog.'),
('SQL is a powerful language for querying databases.'),
('PostgreSQL is an open-source relational database management system.');
"""

cur = conn.cursor()
cur.execute(query_create_table)
cur.execute(query_add_data)
conn.commit()
cur.close()

In [94]:
# query = """
# SELECT *
# FROM sample_text_data
# WHERE text_content like "querying"
# """

# query = """
# SELECT * 
# FROM customer_reviews
# WHERE MATCH (review_text) AGAINST ('SQL');
# """   

query = '''
-- Convert the MySQL query to PostgreSQL format
SELECT * 
FROM sample_text_data
WHERE to_tsvector('english', text_content) @@ to_tsquery('english', 'SQL');
'''

cur = conn.cursor()
cur.execute(query)
cur.fetchall()

[(3, 'SQL is a powerful language for querying databases.')]

In [93]:
conn.commit()
cur.close()

In [96]:
query = '''
-- Convert the MySQL query to PostgreSQL format
SELECT * 
FROM sample_text_data
WHERE text_content like '%SQL%'
'''

cur = conn.cursor()
cur.execute(query)
cur.fetchall()

[(3, 'SQL is a powerful language for querying databases.'),
 (4, 'PostgreSQL is an open-source relational database management system.')]

In [174]:
query = """
CREATE TABLE employees (
    id INT PRIMARY KEY,
    name VARCHAR(100),
    department VARCHAR(100),
    salary DECIMAL(10, 2)
);
"""

cur = conn.cursor()
cur.execute(query)
conn.commit()
cur.close()

query = """
INSERT INTO employees (id, name, department, salary) VALUES
(1, 'John', 'HR', 50000),
(2, 'Emily', 'IT', 60000),
(3, 'Michael', 'Sales', 55000),
(4, 'Jessica', 'Marketing', 52000);
"""

cur = conn.cursor()
cur.execute(query)
conn.commit()
cur.close()

In [111]:
# create a table from select and CREATE TABLE AS
query = """
CREATE TABLE temp_employees AS SELECT * FROM employees WHERE id != 3;
"""

cur = conn.cursor()
cur.execute(query)
conn.commit()
cur.close()

In [177]:
conn.commit()
cur.close()

In [178]:
query = "ALTER TABLE employees DROP COLUMN new_column"
# query = "ALTER TABLE employees ADD COLUMN new_column INT "

# query = """
# INSERT INTO employees (id,new_column) 
# VALUES (1, 1),
#        (2, 2),
#        (3, 3),
#        (4, 4);
# """

# query = """
# UPDATE employees SET new_column = 1 WHERE id = 1
# """

# query = """
# SELECT *
# FROM employees
# """

def make_query(query):

    cur = conn.cursor()

    try:
        if("SELECT" in query.split() and query.split()[0] == "SELECT"):
            cur.execute(query)
            data = cur.fetchall()
            for row in data:
                print(row)
        else:
            cur = conn.cursor()
            cur.execute(query)
            conn.commit()
    except:
        print('cant query')
        conn.commit()

    cur.close()

make_query(query)

# constraints
# PRIMARY KEY
# UNIQUE
# FOREIGN KEY
# NOT NULL
# CHECK

# table constraints
# - PRIMAR  KEY (unique index + NOT NULL)
# - FOREIGN KEY (added index must be in REFERENCE table
# - UNIQUE  (contains unique values)
# - NOT NULL (cannot be NULL)
# - CHECK (check certain condition eg. ALTER TABLE ADD CONSTRAINT column column > 0




In [179]:
# doesn't work for column imports
# insert into (new row)
query = """
INSERT INTO employees (new_column) 
VALUES (1),
       (2),
       (3),
       (4);
"""

# insert value for each column index
# update existing values
# query = """
# UPDATE employees SET new_column = 1 WHERE id = 1;
# UPDATE employees SET new_column = 2 WHERE id = 2;
# UPDATE employees SET new_column = 3 WHERE id = 3;
# UPDATE employees SET new_column = 4 WHERE id = 4;
# """

# query = """DROP TABLE employees"""

cur = conn.cursor()
cur.execute(query)
conn.commit()

UndefinedColumn: column "new_column" of relation "employees" does not exist
LINE 2: INSERT INTO employees (new_column) 
                               ^
